Making Shor's algo scalable (Starting with preexisiting hard coded Shors from textbook)


### Goal
Refactor the Qiskit textbook implementation of Shor’s algorithm into a **scalable, parameterized architecture**.
“Scalable” means the algorithm and code structure scale with `log(N)` and are not hardcoded for small demo values, even if large `N` cannot yet run on current quantum hardware.
## TODO Roadmap
---
### Task 1 (DONE) — Remove fixed / hardcoded N
**What to do**
- Remove any assumptions that `N` is a specific value (e.g., `N = 15`).
- Make `N` an explicit input to the algorithm.
- Compute the bit-length `n = ceil(log2(N))`.
- Size all quantum registers based on `n`.

**Why**
Hardcoding `N` makes the algorithm non-scalable. Shor’s resource usage must scale with the number of bits, not a fixed demo value.

---

### Task 2 (DONE) — Replace hardcoded U with general modular exponentiation
**What to do**
- Identify where the modular multiplication unitary `U` is manually constructed.
- Remove the hardcoded implementation.
- Replace it with a general **modular exponentiation structure** using:
  - repeated squaring
  - controlled modular multiplication (placeholders acceptable)

**Why**
Hardcoded `U` gates only work for tiny values of `N`. Modular exponentiation must be parameterized by `N` and `n` for scalability.

---

### Task 3 (DONE) — Implement reversible arithmetic primitives
**What to do**
- Implement reusable reversible circuits for:
  - addition
  - modular addition
  - modular multiplication
- Ensure all ancilla qubits are uncomputed.

**Why**
Modular exponentiation depends entirely on reversible arithmetic. This is the main depth and qubit bottleneck in Shor’s algorithm.

---

### Task 4 (DONE) — Use qubit-efficient phase estimation
**What to do**
- Replace the full Quantum Fourier Transform (QFT) with an iterative or semiclassical phase estimation approach.

**Why**
Reduces qubit count and improves compatibility with near-term hardware while preserving correctness.

---

### Task 5 (DONE) — Separate circuit construction from execution
**What to do**
- Isolate circuit-building logic from execution logic.
- Ensure circuits can be:
  - simulated (small `N`)
  - executed on hardware (very small `N`)
  - analyzed without execution (large `N`)

**Why**
Scalable algorithms must survive changes in hardware without rewrites.

---

### Task 6 (DONE) — Add hardware-aware transpilation
**What to do**
- Transpile circuits for a chosen IBM backend.
- Track circuit depth, two-qubit gate count, and layout changes.

**Why**
Real quantum hardware has connectivity and gate constraints that strongly affect feasibility.

---

### Task 7 (DONE) 
— Enable hardware execution for small N
**What to do**
- Run the scalable Shor circuit for very small `N` on real IBM hardware.
- Collect and post-process measurement results.

**Why**
Demonstrates correctness and hardware compatibility without claiming cryptographic relevance.

---

### Task 8 (DONE) — Add resource analysis and scaling metrics
**What to do**
- Measure and report:
  - number of qubits
  - circuit depth
  - two-qubit gate count
- Analyze how these scale with `log(N)`.

**Why**
For large `N`, scalability is demonstrated via resource growth, not execution.


In [3]:

import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

In [4]:
def ripple_carry_adder(n):
    """Simple reversible ripple-carry adder.

    Adds register a into b (b := a + b). a is unchanged.
    Uses an (n+1)-qubit carry register.

    Reversible arithmetic is required in quantum algorithms because
    unitary operations must preserve information (no irreversible
    overwrites).
    """
    a = QuantumRegister(n, 'a')
    b = QuantumRegister(n, 'b')
    c = QuantumRegister(n + 1, 'c')
    qc = QuantumCircuit(a, b, c, name='ADD')
    for i in range(n):
        # Full-adder: carry-out in c[i+1], sum in b[i]
        qc.ccx(a[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i + 1])
        qc.cx(c[i], b[i])
    return qc.to_gate()


def add_const_gate(k, n):
    """In-place add constant k to n-qubit register x using a work register.

    This is reversible but not optimized; internal carries are cleaned
    when the gate is inverted inside higher-level routines.
    """
    x = QuantumRegister(n, 'x')
    kreg = QuantumRegister(n, 'k')
    c = QuantumRegister(n + 1, 'c')
    qc = QuantumCircuit(x, kreg, c, name=f'ADD_{k}')
    # Prepare |k> in kreg (classical constant)
    for i in range(n):
        if (k >> i) & 1:
            qc.x(kreg[i])
    qc.append(ripple_carry_adder(n), list(kreg) + list(x) + list(c))
    # Unprepare kreg
    for i in range(n):
        if (k >> i) & 1:
            qc.x(kreg[i])
    return qc.to_gate()


def modular_add_const_gate(k, N, n):
    """Add constant k modulo N to an n-qubit register (reversible, not optimized).

    Modular addition is a building block for modular multiplication,
    which in turn dominates the depth of Shor's algorithm.
    """
    x = QuantumRegister(n, 'x')
    kreg = QuantumRegister(n, 'k')
    c = QuantumRegister(n + 1, 'c')
    flag = QuantumRegister(1, 'flag')
    qc = QuantumCircuit(x, kreg, c, flag, name=f'ADD_{k}_MOD_{N}')

    # 1) x = x + k
    qc.append(add_const_gate(k, n), list(x) + list(kreg) + list(c))

    # 2) x = x - N (by adding 2^n - N), capture carry-out in flag
    two_pow_n = 1 << n
    qc.append(add_const_gate(two_pow_n - N, n), list(x) + list(kreg) + list(c))
    qc.cx(c[n], flag[0])

    # 3) If we underflowed (flag == 0), add N back
    qc.x(flag[0])
    qc.append(add_const_gate(N, n).control(), [flag[0]] + list(x) + list(kreg) + list(c))
    qc.x(flag[0])

    return qc.to_gate()


def modinv(a, N):
    """Classical modular inverse for use in reversible uncomputation.
    Returns a^{-1} mod N if it exists.
    """
    try:
        return pow(a, -1, N)
    except TypeError:
        # Fallback for older Python versions
        t, new_t = 0, 1
        r, new_r = N, a
        while new_r != 0:
            q = r // new_r
            t, new_t = new_t, t - q * new_t
            r, new_r = new_r, r - q * new_r
        if r > 1:
            raise ValueError('a has no modular inverse')
        if t < 0:
            t += N
        return t


def modular_multiply_const(a, N, n):
    """In-place modular multiplication by a (correct but not optimized).

    This uses reversible modular addition and a clean uncompute step
    based on the modular inverse of a. The structure is scalable
    (polynomial in n = log2(N)) but not hardware-efficient yet.
    """
    x = QuantumRegister(n, 'x')
    acc = QuantumRegister(n, 'acc')
    kreg = QuantumRegister(n, 'k')
    c = QuantumRegister(n + 1, 'c')
    flag = QuantumRegister(1, 'flag')
    qc = QuantumCircuit(x, acc, kreg, c, flag, name=f'MUL_{a}_MOD_{N}')

    # Compute acc = a * x (mod N) using repeated squaring additions
    for i in range(n):
        k = (a * (2 ** i)) % N
        qc.append(modular_add_const_gate(k, N, n).control(),
                  [x[i]] + list(acc) + list(kreg) + list(c) + list(flag))

    # Swap acc into x to make the operation in-place on x
    for i in range(n):
        qc.swap(x[i], acc[i])

    # Uncompute acc using a^{-1} so all ancillas return to |0>
    a_inv = modinv(a, N)
    for i in reversed(range(n)):
        k_inv = (a_inv * (2 ** i)) % N
        qc.append(modular_add_const_gate(k_inv, N, n).inverse().control(),
                  [x[i]] + list(acc) + list(kreg) + list(c) + list(flag))

    return qc.to_gate()


def controlled_modular_multiply(a, N, n):
    """Controlled modular multiplication by a mod N.
    The control qubit comes from the phase estimation register.
    """
    return modular_multiply_const(a, N, n).control()


def a2jmodN(a, j, N):
    """Compute a^(2^j) (mod N) by repeated squaring (classical helper).

    This mirrors the structure used in the quantum circuit: each counting
    qubit controls a multiplication by a^(2^j) mod N.
    """
    for _ in range(j):
        a = (a * a) % N
    return a


def c_amodN(a, power, N, n):
    """Controlled multiplication by a^(2^power) mod N.

    Modular exponentiation is the heart of Shor's algorithm. The
    repeated-squaring structure (powers of two) is what makes the
    circuit scale with log(N) instead of being hardcoded for a
    particular small N.
    """
    a_power = a2jmodN(a, power, N)
    return controlled_modular_multiply(a_power, N, n)


def choose_coprime_base(N, rng=None):
    """Pick a random a in [2, N-1] that is coprime to N.

    Returns (a, gcd) so callers can short-circuit if gcd != 1.
    """
    if rng is None:
        rng = np.random.default_rng()
    while True:
        a = int(rng.integers(2, N))
        g = gcd(a, N)
        if g == 1:
            return a, g
        # If g is a non-trivial factor, caller can use it directly.
        return a, g


# Specify variables
# Set N in one place for programmatic sweeps
N = 15
n = int(np.ceil(np.log2(N)))  # register size scales with log2(N)
N_COUNT = 2 * n  # typical choice for phase estimation
rng = np.random.default_rng(1)
a, g = choose_coprime_base(N, rng=rng)
if g != 1:
    print(f"Non-trivial factor found classically: {g}")


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ
    """
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFTdagger"
    return qc

def build_qpe_circuit(a, N, N_COUNT=None):
    """Construct the QPE circuit for Shor's algorithm (no execution).

    Returns (qc, N_COUNT).
    """
    n = int(np.ceil(np.log2(N)))
    if N_COUNT is None:
        N_COUNT = 2 * n
    # Create QuantumCircuit with N_COUNT counting qubits
    # plus work + ancilla qubits for modular multiplication
    # Work register: n qubits
    # Ancillas: acc (n), k (n), carry (n+1), flag (1)
    qc = QuantumCircuit(N_COUNT + 4 * n + 2, N_COUNT)
    # Initialize counting qubits in state |+>
    for q in range(N_COUNT):
        qc.h(q)
    # Work register in state |1>
    qc.x(N_COUNT)
    # Modular exponentiation via repeated squaring
    for q in range(N_COUNT):
        x = [N_COUNT + i for i in range(n)]
        acc = [N_COUNT + n + i for i in range(n)]
        kreg = [N_COUNT + 2 * n + i for i in range(n)]
        carry = [N_COUNT + 3 * n + i for i in range(n + 1)]
        flag = N_COUNT + 4 * n + 1
        qc.append(c_amodN(a, q, N, n),
                 [q] + x + acc + kreg + carry + [flag])
    # Inverse QFT and measurements
    qc.append(qft_dagger(N_COUNT), range(N_COUNT))
    qc.measure(range(N_COUNT), range(N_COUNT))
    return qc, N_COUNT


In [8]:
# Build QPE circuit (construction only; execution is separate)
qc, N_COUNT = build_qpe_circuit(a, N, N_COUNT)
qc.draw(fold=-1)  # -1 means 'do not fold'


┌───┐                                                                                                                                                ┌────────────┐┌─┐                     
 q_0: ┤ H ├────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0           ├┤M├─────────────────────
      ├───┤        │                                                                                                                                       │            │└╥┘┌─┐                  
 q_1: ┤ H ├────────┼─────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤1           ├─╫─┤M├──────────────────
      ├───┤        │                 │                                                                                                                     │            │ ║ └╥┘┌─┐               
 q_2: ┤ H ├────────┼─────────────────┼─────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────┤2           ├─╫──╫─┤M├───────────────
      ├───┤        │                 │                 │                                                                                                   │            │ ║  ║ └╥┘┌─┐            
 q_3: ┤ H ├────────┼─────────────────┼─────────────────┼─────────────────■─────────────────────────────────────────────────────────────────────────────────┤3           ├─╫──╫──╫─┤M├────────────
      ├───┤        │                 │                 │                 │                                                                                 │  QFTdagger │ ║  ║  ║ └╥┘┌─┐         
 q_4: ┤ H ├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────■───────────────────────────────────────────────────────────────┤4           ├─╫──╫──╫──╫─┤M├─────────
      ├───┤        │                 │                 │                 │                 │                                                               │            │ ║  ║  ║  ║ └╥┘┌─┐      
 q_5: ┤ H ├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────■─────────────────────────────────────────────┤5           ├─╫──╫──╫──╫──╫─┤M├──────
      ├───┤        │                 │                 │                 │                 │                 │                                             │            │ ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_6: ┤ H ├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────■───────────────────────────┤6           ├─╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤        │                 │                 │                 │                 │                 │                 │                           │            │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_7: ┤ H ├────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────■─────────┤7           ├─╫──╫──╫──╫──╫──╫──╫─┤M├
      ├───┤┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐┌───────┴────────┐└────────────┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
 q_8: ┤ X ├┤0               ├┤0               ├┤0               ├┤0               ├┤0               ├┤0               ├┤0               ├┤0               ├───────────────╫──╫──╫──╫──╫──╫──╫──╫─
      └───┘│                ││                ││                ││                ││                ││                ││                ││                │               ║  ║  ║  ║  ║  ║  ║  ║ 
 q_9: ─────┤1               ├┤1               ├┤1               ├┤1               ├┤1               ├┤1               ├┤1               ├┤1               ├───────────────╫──╫──╫──╫──╫──╫──╫──╫─
           │                ││                ││                ││                ││                ││                ││      

In [9]:
backend = AerSimulator()
qc_t = transpile(qc, backend)
job = backend.run(qc_t, shots=128)
result = job.result()
counts = result.get_counts(qc_t)
plot_histogram(counts)
rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**N_COUNT)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)",
                 f"{decimal}/{2**N_COUNT} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)


             Register Output           Phase
0   01110101(bin) = 117(dec)  117/256 = 0.46
1   00010101(bin) =  21(dec)   21/256 = 0.08
2   00101001(bin) =  41(dec)   41/256 = 0.16
3   01100111(bin) = 103(dec)  103/256 = 0.40
4   01100101(bin) = 101(dec)  101/256 = 0.39
5   10110011(bin) = 179(dec)  179/256 = 0.70
6   11100111(bin) = 231(dec)  231/256 = 0.90
7   00011101(bin) =  29(dec)   29/256 = 0.11
8   00110011(bin) =  51(dec)   51/256 = 0.20
9   10011011(bin) = 155(dec)  155/256 = 0.61
10  00001011(bin) =  11(dec)   11/256 = 0.04
11  01001101(bin) =  77(dec)   77/256 = 0.30
12  10001101(bin) = 141(dec)  141/256 = 0.55
13  01010011(bin) =  83(dec)   83/256 = 0.32
14  10010011(bin) = 147(dec)  147/256 = 0.57
15  11110101(bin) = 245(dec)  245/256 = 0.96
16  11010101(bin) = 213(dec)  213/256 = 0.83
17  10101011(bin) = 171(dec)  171/256 = 0.67
18  10010101(bin) = 149(dec)  149/256 = 0.58
19  11101001(bin) = 233(dec)  233/256 = 0.91
20  01010101(bin) =  85(dec)   85/256 = 0.33
21  101011

In [10]:
Fraction(0.666)

# Get fraction that most closely resembles 0.666
# with denominator < N
Fraction(0.666).limit_denominator(N)
rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(N)
    rows.append([phase,
                 f"{frac.numerator}/{frac.denominator}",
                 frac.denominator])
# Print as a table
headers=["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)


       Phase Fraction  Guess for r
0   0.457031     5/11           11
1   0.082031     1/12           12
2   0.160156     2/13           13
3   0.402344      2/5            5
4   0.394531      2/5            5
5   0.699219     7/10           10
6   0.902344     9/10           10
7   0.113281      1/9            9
8   0.199219      1/5            5
9   0.605469      3/5            5
10  0.042969     1/15           15
11  0.300781     3/10           10
12  0.550781      5/9            9
13  0.324219      1/3            3
14  0.574219      4/7            7
15  0.957031    14/15           15
16  0.832031      5/6            6
17  0.667969      2/3            3
18  0.582031     7/12           12
19  0.910156    10/11           11
20  0.332031      1/3            3
21  0.675781      2/3            3
22  0.144531      1/7            7
23  0.500000      1/2            2
24  0.000000      0/1            1
25  0.152344     2/13           13
26  0.589844     7/12           12
27  0.417969     5/1

In [11]:
a2jmodN(7, 2049, 53)


rng = np.random.default_rng(1)  # reproducible base selection
a, g = choose_coprime_base(N, rng=rng)
print(a)

from math import gcd # greatest common divisor
gcd(a, N)
def qpe_amodN(a, N, shots=128, backend=None, rng=None):
    """Performs quantum phase estimation on the operation a*r mod N.
    Args:
        a (int): This is 'a' in a*r mod N
        N (int): Composite to factor
        shots (int): Number of samples for phase estimation
        backend: Qiskit backend (simulator or hardware).
    Returns:
        float: Estimate of the phase
    """
    qc, N_COUNT = build_qpe_circuit(a, N)
    # Execute on provided backend (simulator or hardware)
    if backend is None:
        backend = AerSimulator()
    # `memory=True` tells the backend to save each measurement in a list
    job = backend.run(transpile(qc, backend), shots=shots, memory=True)
    readings = job.result().get_memory()
    if rng is None:
        rng = np.random.default_rng()
    reading = readings[int(rng.integers(0, len(readings)))]
    print("Register Reading: " + reading)
    phase = int(reading,2)/(2**N_COUNT)
    print(f"Corresponding Phase: {phase}")
    return phase

phase = qpe_amodN(a, N, shots=128, rng=rng) # Phase = s/r
Fraction(phase).limit_denominator(N)

frac = Fraction(phase).limit_denominator(N)
s, r = frac.numerator, frac.denominator
print(r)
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)


8
Register Reading: 01000000
Corresponding Phase: 0.25
4
[3, 5]


In [12]:
a, g = choose_coprime_base(N, rng=rng)
FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nATTEMPT {ATTEMPT}:")
    if g != 1:
        print(f"*** Non-trivial factor found classically: {g} ***")
        FACTOR_FOUND = True
        break
    phase = qpe_amodN(a, N, shots=128, rng=rng) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    print(f"Result: r = {r}")
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} +- 1 , N)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0:
                # Guess is a factor!
                print(f"*** Non-trivial factor found: {guess} ***")
                FACTOR_FOUND = True
    if not FACTOR_FOUND:
        a, g = choose_coprime_base(N, rng=rng)


# Loop terminates when a non-trivial factor is found
assert FACTOR_FOUND



ATTEMPT 1:
Register Reading: 10101101
Corresponding Phase: 0.67578125
Result: r = 3
Guessed Factors: 5 and 3
*** Non-trivial factor found: 5 ***
*** Non-trivial factor found: 3 ***
